## <b> Project Title : Seoul Bike Sharing Demand Prediction </b>
**Individual Project:**



###**Github Link:**

https://github.com/RishiAtWork1/Regression-Capstone-Project-2



### **Problem Statement:**


**Currently Rental bikes are introduced in many urban cities for the enhancement of mobility comfort. It is important to make the rental bike available and accessible to the public at the right time as it lessens the waiting time. Eventually, providing the city with a stable supply of rental bikes becomes a major concern. The crucial part is the prediction of bike count required at each hour for the stable supply of rental bikes.**

### <b> Data Description </b>

##### <b> The dataset contains weather information (Temperature, Humidity, Windspeed, Visibility, Dewpoint, Solar radiation, Snowfall, Rainfall), the number of bikes rented per hour and date information.</b>


### <b>Attribute Information: </b>

* ##### Date : year-month-day
* ##### Rented Bike count - Count of bikes rented at each hour
* ##### Hour - Hour of the day
* ##### Temperature-Temperature in Celsius
* ##### Humidity - %
* ##### Windspeed - m/s
* ##### Visibility - 10m
* ##### Dew point temperature - Celsius
* ##### Solar radiation - MJ/m2
* ##### Rainfall - mm
* ##### Snowfall - cm
* ##### Seasons - Winter, Spring, Summer, Autumn
* ##### Holiday - Holiday/No holiday
* ##### Functional Day - NoFunc(Non Functional Hours), Fun(Functional hours)

### **Importing the libraries and the dataset**

In [ ]:
#let's import the modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from datetime import datetime
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

###**Mount the drive and import the datset**

In [ ]:
#let's mount the google drive for import the dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

#load and read the seol bike data set from drive
bike_df = pd.read_csv('/content/drive/MyDrive/SeoulBikeData.csv',encoding='latin')
bike_df.head()

###**Features description**

**Breakdown of Our Features:**

**Date** : *The date of the day, during 365 days from 01/12/2017 to 30/11/2018, formating in DD/MM/YYYY, type : str*, we need to convert into datetime format.

**Rented Bike Count** : *Number of rented bikes per hour which our dependent variable and we need to predict that, type : int*

**Hour**: *The hour of the day, starting from 0-23 it's in a digital time format, type : int, we need to convert it into category data type.*

**Temperature(°C)**: *Temperature in Celsius, type : Float*

**Humidity(%)**: *Humidity in the air in %, type : int*

**Wind speed (m/s)** : *Speed of the wind in m/s, type : Float*

**Visibility (10m)**: *Visibility in m, type : int*

**Dew point temperature(°C)**: *Temperature at the beggining of the day, type : Float*

**Solar Radiation (MJ/m2)**: *Sun contribution, type : Float*

**Rainfall(mm)**: *Amount of raining in mm, type : Float*

**Snowfall (cm)**: *Amount of snowing in cm, type : Float*

**Seasons**: *Season of the year, type : str, there are only 4 season's in data *.

**Holiday**: *If the day  is holiday period or not, type: str*

**Functioning Day**: *If the day is a Functioning Day or not, type : str*


#**Understand the data**

In [ ]:
#Getting the shape of dataset with rows and columns
print(bike_df.shape)

In [ ]:
#Getting all the columns
print("Features of the dataset:")
bike_df.columns

In [ ]:
#check details about the data set
bike_df.info()

In [ ]:
#print the unique value
bike_df.nunique()

In [ ]:
# checking the null value counts
print(bike_df.isnull().sum())

In [ ]:
# Checking Duplicate Values
value=len(bike_df[bike_df.duplicated()])
print("The number of duplicate values in the data set is = ",value)

In [ ]:
#Looking for the description of the dataset to get insights of the data
bike_df.describe().astype(int).T

* This Dataset contains **8760 lines** and **14 columns**.
* In a day we have **24 hours** and we have **365 days** a year so  **365*24 = 8760**, which represents the number of line in the dataset.means we have the data of whole year.

In [ ]:
#Rename the complex columns name
bike_df=bike_df.rename(columns={'Rented Bike Count':'Rented_Bike_Count','Temperature(°C)':'Temperature','Humidity(%)':'Humidity',
                                'Wind speed (m/s)':'Wind_speed','Visibility (10m)':'Visibility',
                                'Dew point temperature(°C)':'Dew_point_temperature','Solar Radiation (MJ/m2)':'Solar_Radiation',
                                'Rainfall(mm)':'Rainfall','Snowfall (cm)':'Snowfall','Functioning Day':'Functioning_Day'})

###**Breaking date column**
* Spliting Date column which is in string format into year, month, day as a category data type.

In [ ]:

# Changing the "Date" column into three "year","month","day" column
bike_df['Date'] = bike_df['Date'].apply(lambda x:
                                    dt.datetime.strptime(x,"%d/%m/%Y"))
bike_df['year'] = bike_df['Date'].dt.year
bike_df['month'] = bike_df['Date'].dt.month
bike_df['day'] = bike_df['Date'].dt.day_name()


In [ ]:
#Observing the pattern of data
bike_df.iloc[30:35,:]

In [ ]:

#creating a new column of "weekdays_weekend" and drop the column "Date","day","year"
bike_df['weekdays_weekend']=bike_df['day'].apply(lambda x : 1 if x=='Saturday' or x=='Sunday' else 0 )
bike_df=bike_df.drop(columns=['Date','day','year'],axis=1)

In [ ]:
bike_df.head()

In [ ]:
#counting the value
print(bike_df['Functioning_Day'].value_counts())
print(bike_df['weekdays_weekend'].value_counts())

* So we convert the "date" column into 3 different column i.e "year","month","day".
* The "year" column is basically contain **from 2017 december to 2018 november i.e. one year** **we don't need the "year" column so we drop it**.
* The other column "day", it contains the each day of the month,we don't need each day of each month data but we need the data about, **if a day is a weekday or a weekend** so we convert it into this format and **drop the "day" column**.

#**Exploratory data analysis**

### **Univariate Analysis**
* ***The key objective of Univariate analysis is to simply describe the data to find patterns within the data.***
* ***we analyse our dependent variable,A dependent variable is a variable whose value will change depending on the value of another variable.***
* ***Our dependent variable is "Rented Bike Count" so we need to analyse this column with the other columns by using some visualisation plot.first we analyze the category data type then we proceed with the numerical data type***

####**Analysis of Categorical data by visualization**
* ***Month***      
* ***weekdays_weekend***
* ***Hour***
* ***Functioning Day***
* ***Season***
* ***Holiday***


In [ ]:
#categorical_features
categorical_column = ['Hour', 'month', 'weekdays_weekend','Seasons','Holiday','Functioning_Day']

In [ ]:
#Visualization by barchart and boxplot of categorical variables
plt.figure(figsize=(15,10))
for n,col in enumerate(categorical_column):
  plt.subplot(2,3,n+1)
  plt.title(col)
  sns.barplot(data=bike_df,x=col,y='Rented_Bike_Count')
  plt.tight_layout();

#getting boxplot for categorical features vs rented bike count
plt.figure(figsize=(15,10))
for n,col in enumerate(categorical_column):
  plt.subplot(2,3,n+1)
  plt.title(col)
  sns.boxplot(data=bike_df,x=col,y='Rented_Bike_Count');
  plt.tight_layout();

#getting countplot for hour
plt.figure(figsize=(12,8))
for n,col in enumerate(['weekdays_weekend','Seasons','Holiday','Functioning_Day']):
  plt.subplot(2,2,n+1)
  plt.title(col)
  sns.pointplot(data=bike_df,x='Hour',y='Rented_Bike_Count',hue=col)
  plt.tight_layout();

#getting countplot for month
plt.figure(figsize=(16,4))
for n,col in enumerate(['weekdays_weekend','Seasons','Holiday','Functioning_Day']):
  plt.subplot(1,4,n+1)
  plt.title(col)
  sns.pointplot(data=bike_df,x='month',y='Rented_Bike_Count',hue=col)
  plt.tight_layout();


* ***generally people use rented bikes during their working hour from 7am to 9am and 5pm to 9pm.***   

* ***the demand of the rented bike is high in May, June, July, August, September as compare to other months.these months are comes inside the summer season.***

* ***in the week days(Peak Time: 7 am to 9 am, 5 pm to 9 pm) the demand of the bike higher because of the office.***

* **on weekend days, the demand of rented bikes are very low specially in the morning hour but from 4 pm to 8 pm the demand slightly increases.**

* ***Peoples use rented bikes mostly on functioning day.***

* ***highest rented bike : in summer season(peak time: 7am-9am and 5pm-9pm), lowest rented bike : in winter season, highest rented bike on holidays : in May-June month, lowest rented bike on holidays : in December-January-February month***

* ***the use of rented bike is highest when there is no holiday(peak time: 2pm-8pm)***


####**Analysis of Numerical data by visualization**
* ***Numerical data is a data type expressed in numbers, rather than natural language description. Sometimes called quantitative data, numerical data is always collected in number form. Numerical data differentiates itself from other number form data types with its ability to carry out arithmetic operations with these numbers.***

In [ ]:
#making list of continuous variables
numerical_features = ['Temperature', 'Humidity', 'Wind_speed',
       'Visibility', 'Dew_point_temperature', 'Solar_Radiation', 'Rainfall',
       'Snowfall']

###**Numerical vs rented_bike_count**

In [ ]:
# making function to print all numerical features
plt.figure(figsize=(16,8))
for n,col in enumerate(numerical_features):
  plt.subplot(2,4,n+1)
  bike_df.groupby(col).mean()['Rented_Bike_Count'].plot();
  plt.tight_layout();

* From the above plot we see that people like to ride bikes when it is pretty hot around 25°C average.
* We can see from the above plot that the **demand of rented bike is uniformly distributed** **despite of wind speed** but when the speed of wind was 7 m/s then the demand of bike also increase that clearly means **peoples love to ride bikes when its little windy**.
* from the above plot we see that, the **amount of rented bikes is huge, when there is solar radiation**. when there is increase in solar radiation, there is increase in rented bike.
* we can see from above plot, **when there is no rain, rented bike count is huge**, but when there is rain, we can see increase in bike rented only at certain points, maybe it is when people are going to home from office on weekdays. but otherwise,there is no huge spike in the rented bike.
* We can see from the plot that, on the y-axis, **the amount of rented bike is very low When we have more than 4 cm of snow**, the bike rents is much lower

###**Regression plot**
**The regression plots in seaborn are primarily intended to add a visual guide that helps to emphasize patterns in a dataset during exploratory data analyses. Regression plots as the name suggests creates a regression line between 2 parameters and helps to visualize their linear relationships.**

In [ ]:
#printing the regression plot for all the numerical features
plt.figure(figsize=(16,8))
for n,col in enumerate(numerical_features):
  plt.subplot(2,4,n+1)
  sns.regplot(x=bike_df[col],y=bike_df['Rented_Bike_Count'],scatter_kws={"color": 'orange'}, line_kws={"color": "black"});
  plt.title(col)
  plt.tight_layout();


* From the above regression plot of all numerical features we see that the columns  **Temperature, Wind_speed, Solar_Radiation Snowfall are positively relation to the target variable**.which means the rented bike count increases with increase of these features.
* **Rainfall, Snowfall, Humidity these features are negatively related with the target variable** which means the rented bike count decreases when these features increase.

###**Checking skewness and outliers of the continuous Variable**

In [ ]:
#checking distribution and boxplot of continuous features
plt.figure(figsize=(16,8))
for n,col in enumerate(numerical_features):
  plt.subplot(2,4,n+1)
  plt.xlabel(col)
  plt.ylabel('Density')
  ax=sns.distplot(np.sqrt(bike_df[col]),hist=True ,color="y")
  ax.axvline((bike_df[col]).mean(),color='red')
  ax.axvline((bike_df[col]).median(),color='black')
  plt.title(col)
  plt.tight_layout();

plt.figure(figsize=(16,8))
for n,col in enumerate(numerical_features):
    plt.subplot(2,4,n+1)
    plt.ylabel(col)
    sns.boxplot(x=(bike_df[col]))
    plt.tight_layout();

* The above graph shows that Rented_Bike_Count,Visibility,Solar_Radiation,Rainfall,Snowfall has **highly skewed**. Since the assumption of linear regression is that 'the **distribution of dependent and independent variable has to be normal**', so we should perform some operation to make it normal.
* From boxplot, we have detected outliers in Rented Bike Count, Snowfall,Rainfall, Solar radiation, Wind speed column.

### **Methods to treat outliers:**
* **Trimming/Remove the outliers:** In this technique, we remove the outliers from the dataset. Although it is not a good practice to follow.
* **Quantile based flooring and capping:** In this technique, the outlier is capped at a certain value above the 90th percentile value or floored at a factor below the 10th percentile value.
* **Mean/Median imputation:** As the mean value is highly influenced by the outliers, it is advised to replace the outliers with the median value.

###**Method to treat Skewness:**
* **1.log transformation:** transform skewed distribution to a normal distribution.
* **2.Remove outliers**
* **3.Normalize (min-max)**
* **4.Cube root:** when values are too large. Can be applied on negative values.
* **5.Square root:** applied only to positive values.
* **6.Reciprocal**
* **7.Square: apply on left skew**
* **Skewness in target variable:** Use undersampling, oversampling or SMOTE

In [ ]:
#Handling Skewness in the data
bike_df['Rented_Bike_Count']=np.sqrt(bike_df['Rented_Bike_Count'])
bike_df['Snowfall']=np.cbrt(bike_df['Snowfall'])
bike_df['Rainfall']=np.cbrt(bike_df['Rainfall'])
bike_df['Solar_Radiation']=np.cbrt(bike_df['Solar_Radiation'])
bike_df['Humidity']=np.sqrt(bike_df['Humidity'])

In [ ]:
#checking distribution and boxplot of continuous features after applying some algorithm
plt.figure(figsize=(16,8))
for n,col in enumerate(numerical_features):
  plt.subplot(2,4,n+1)
  plt.xlabel(col)
  plt.ylabel('Density')
  ax=sns.distplot(np.sqrt(bike_df[col]),hist=True ,color="y")
  ax.axvline((bike_df[col]).mean(),color='red')
  ax.axvline((bike_df[col]).median(),color='black')
  plt.title(col)
  plt.tight_layout();

plt.figure(figsize=(16,8))
for n,col in enumerate(numerical_features):
    plt.subplot(2,4,n+1)
    plt.ylabel(col)
    sns.boxplot(x=(bike_df[col]))
    plt.tight_layout();

* Since we have generic rule of applying Square root for the skewed variable in order to make it normal .**After applying Square root to the skewed Rented Bike Count, here we get almost normal distribution**.
* After applying Square root to the Rented Bike Count column, we find that there is no outliers present.

#**Feature selection**

###**Correlation Map**

In [ ]:
# correlation map
plt.figure(figsize=(16,15))
corr = bike_df.corr()
sns.heatmap(corr,annot=True,fmt='.2f',annot_kws={'size':15})
plt.title('correlation between features');

***We can observe on the heatmap that on the target variable line the most positively correlated variables to the bike rent are :***

* the temperature
* the dew point temperature
* the solar radiation
* Hour

***And most negatively correlated variables are:***
* Humidity
* Rainfall
* weekdays or weekends


from above correlation map, we can see that, there is **high correlation between 'Dew Point Temperature' and Temperature**. here we are featuring the best suitable model,sp we have to drop either one of the feature i.e. either Temperature or Dew Point Temperature.


###**Checking Multicollinearity using VIF**

In [ ]:
#Defining Variance Inflaition Factor
from statsmodels.stats.outliers_influence import variance_inflation_factor

def cal_vif(x):
  vif = pd.DataFrame()
  vif['variables'] = x.columns
  vif['VIF'] = [variance_inflation_factor(x.values,i) for i in range(x.shape[1])]
  return vif

In [ ]:
#Checking Variance Inflaition Factor
VIF_table = cal_vif(bike_df[[i for i in bike_df.describe().columns if i not in ['Rented_Bike_Count']]])
VIF_table

**1. Variance Inflation Factor(VIF)**

– If VIF=1; No multicollinearity

– If VIF=<5; Low multicollinearity or moderately correlated

– If VIF=>5; High multicollinearity or highly correlated

**2. Tolerance(Reciprocal of VIF)**

– If VIF is high then tolerance will be low i.e, high multicollinearity.

– If VIF is low the tolerance will be high i.e, low multicollinearity.

In [ ]:
# dropping dew point temperature column which has high VIF
bike_df = bike_df.drop(['Dew_point_temperature'],axis=1)

In [ ]:
#Checking Variance Inflaition Factor
VIF_table = cal_vif(bike_df[[i for i in bike_df.describe().columns if i not in ['Rented_Bike_Count']]])
VIF_table

In [ ]:
# dropping dew point temperature column which has high VIF
bike_df = bike_df.drop(['Humidity'],axis=1)

In [ ]:
#removing visibility and dewpoint temperature from numerical feature column
numerical_features.remove('Dew_point_temperature')
numerical_features.remove('Humidity')

In [ ]:
#Checking Variance Inflaition Factor
VIF_table = cal_vif(bike_df[[i for i in bike_df.describe().columns if i not in ['Rented_Bike_Count']]])
VIF_table

Since VIF of all variables is <5, we can go with remaining variables.

In [ ]:
#Getting columns
bike_df.columns

In [ ]:
# description of numerical values
bike_df.describe().astype(float).T

In [ ]:
# description of object values
bike_df.describe(include='O').T

* here we can see that, **highest rented bike** is during **Spring season**.
* here we can see that, **highest rented bike** is when there is **no Holiday and Functioning Day**.

#**Data preprocessing**

###**Create Dummy Variables**
A dataset may contain various type of values, sometimes it consists of categorical values. So, in-order to use those categorical value for programming efficiently we create dummy variables.

In [ ]:
categorical_features = ['Hour','Seasons', 'Holiday', 'Functioning_Day', 'month','weekdays_weekend']
categorical_features

###**one hot encoding**

In [ ]:
#one hot encoding using simple method
bike_df1 = pd.get_dummies(bike_df,columns=['Hour','Seasons', 'Holiday', 'Functioning_Day', 'month','weekdays_weekend'],drop_first=True)

In [ ]:
bike_df1.head()

#**ML model implementation**

###**Train Test Split for regression**
Before, fitting any model it is a rule of thumb to split the dataset into a training and test set. This means some proportions of the data will go into training the model and some portion will be used to evaluate how our model is performing on any unseen data. The proportions may vary from 60:40, 70:30, 75:25 depending on the person but mostly used is 80:20 for training and testing respectively. In this step we will split our data into training and testing set using scikit learn library.

In [ ]:
#Assign the value in X and Y
x = bike_df1.drop(['Rented_Bike_Count'],axis=1)
y = np.sqrt(bike_df1['Rented_Bike_Count'])
y

In [ ]:
#Name of Features
x_columns = x.columns
x_columns

In [ ]:
x.head()

In [ ]:
y.head()

##**Standardising the features**

In [ ]:
#Create test and train data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

In [ ]:
#Standardising the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
#Transformed data
x_train = pd.DataFrame(x_train,columns=x_columns)
x_test = pd.DataFrame(x_test,columns=x_columns)
print(x_train.head())

In [ ]:
bike_df1.columns

###**Function for Graphical representation**

In [ ]:
#function for feature importance
def feature_importance(model):
    try:
      importance = model.feature_importances_
      feature = x_columns
    except:
      importance = np.abs(model.coef_)
      feature = x_columns
    indices = np.argsort(importance)
    indices = indices[20::-1]

    plt.figure(figsize=(12,4))
    plt.barh(range(len(indices)),importance[indices] )
    plt.yticks(range(len(indices)),[feature[i] for i in indices])
    plt.title('Feature Importance')
    plt.show();

##**function for metric values**

In [ ]:
def metric_value(model,x_train,y_train,x_test,y_test):
    #get the y predicted value for train and test dataset
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)

    #Calculating metric value for testing dataset
    #Calculate MSE
    MSE = mean_squared_error(y_train, y_pred_train)
    #calculate RMSE
    RMSE = np.sqrt(MSE)
    #calculate MAE
    MAE = mean_absolute_error(y_train, y_pred_train)
    #calculate r2 and adjusted r2
    r2 = r2_score(y_train, y_pred_train)
    Adjusted_r2 = 1-(((1-r2)*(x_test.shape[0]-1))/(x_test.shape[0]-x_test.shape[1]-1))
    #creating table of metric values
    data_set = [['MAE',round((MAE),3)],['MSE',round((MSE),3)],
                ['RMSE',round((RMSE),3)],['R2_score',round((r2),3)],['Adjusted R2',round((Adjusted_r2),3)]]
    training_df = pd.DataFrame(data_set, columns=['metrics','train_values'])

    #Calculating metric value for testing dataset
    #calculate MSE
    MSE = mean_squared_error(y_test, y_pred_test)
    #calculate RMSE
    RMSE = np.sqrt(MSE)
    #calculate MAE
    MAE = mean_absolute_error(y_test, y_pred_test)
    #calculate r2 and adjusted r2
    r2 = r2_score(y_test, y_pred_test)
    Adjusted_r2 = 1-(((1-r2)*(x_test.shape[0]-1))/(x_test.shape[0]-x_test.shape[1]-1))
    #creating table of metric values
    data_set = [['MAE',round((MAE),3)],['MSE',round((MSE),3)],
                ['RMSE',round((RMSE),3)],['R2_score',round((r2),3)],['Adjusted R2',round((Adjusted_r2),3)]]
    testing_df = pd.DataFrame(data_set, columns=['metrics','test_values'])

    metric_values = training_df.merge(testing_df,how='inner', on='metrics')
    print(metric_values)

    plt.figure(figsize=(4,4))
    ax = metric_values.plot(kind='bar', x='metrics', rot=0)
    plt.title(model)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show();

    ### Heteroscadacity
    residuals = y_test - y_pred_test
    plt.figure(figsize=(9,3))
    #plotting the distribution
    plt.subplot(1,2,1)
    plt.xlabel('residuals')
    sns.distplot(residuals);

    #plotting the scatterplot
    plt.subplot(1,2,2)
    plt.xlabel('scatterplot of residuals')
    plt.scatter(y_pred_test,residuals)
    plt.tight_layout();

    plt.figure(figsize=(12,3))
    plt.plot((y_pred_test)[:100])
    plt.plot((np.array(y_test)[:100]))
    plt.legend(["Predicted","Actual"])
    plt.title('Actual and Predicted Bike Counts')
    plt.show();

    try:
      if model == xgb_model:
        result=pd.DataFrame()
        model = [Linear_Regression,ridge_regression,model,rf_model,gradient_boosting_regressor,xgb_model]
        for i in model:
          result = result.append(metric_values[i],ignore_index=True)
        print(result)
    except:
      pass


##**Linear Regression**

In [ ]:
#import the packages
from sklearn.linear_model import LinearRegression
Linear_Regression = LinearRegression()
Linear_Regression.fit(x_train,y_train)

In [ ]:
#Getting the metric value
metric_value(Linear_Regression,x_train,y_train,x_test,y_test)

In [ ]:
#feature importance
feature_importance(Linear_Regression)

* **Simple Linear Regression Model**
* **R2 score of Train data : 0.861**
* **adjusted R2 score of Train data : 0.857**
* **R2 score of Test data : 0.868**
* **adjusted R2 score of Test data : 0.864**
* **Performance of data: Better**
* To satisfy the regression assumptions and be able to trust the results, **the residuals should have a constant variance**.
* **Important features:** weekdays_weekend,holiday_noholiday,hour:6,13,14,12,19,20,15,22,8


##**Ridge Regression**

In [ ]:
#import the packages
from sklearn.linear_model import Ridge
ridge_regression = Ridge(alpha=0.1)
ridge_regression.fit(x_train,y_train)

In [ ]:
#Getting the metric value
metric_value(ridge_regression,x_train,y_train,x_test,y_test)

In [ ]:
#feature importance
feature_importance(ridge_regression)

* **Ridge Regression Model**
* **R2 score of Train data : 0.861**
* **adjusted R2 score of Train data : 0.857**
* **R2 score of Test data : 0.867**
* **adjusted R2 score of Test data : 0.864**
* **Performance of data: Better**
* To satisfy the regression assumptions and be able to trust the results, **the residuals should have a constant variance**.
* **Important features:** weekdays_weekend,holiday_noholiday,hour:1,16,9, month:3,5,8,9,12

##**Decision Tree with gridsearchcv**

In [ ]:
dt = DecisionTreeRegressor(random_state=0)
dt_params = {'max_depth':np.arange(1,50,2),'min_samples_leaf':np.arange(2,15)}

from sklearn.model_selection import GridSearchCV
gs_dt = GridSearchCV(dt,dt_params,cv=3)
gs_dt.fit(x_train,y_train)
a = gs_dt.best_params_

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(max_depth=a['max_depth'],min_samples_leaf= a['min_samples_leaf'])
model = dtr.fit(x_train,y_train)

In [ ]:
#Getting the metric value
metric_value(model,x_train,y_train,x_test,y_test)

In [ ]:
#feature importance
feature_importance(model)

* **Decision Tree Regression with GridSearchCV Model**
* **R2 score of Train data : 0.98**
* **adjusted R2 score of Train data : 0.98**
* **R2 score of Test data : 0.89**
* **adjusted R2 score of Test data : 0.89**
* **Performance of data: Better**
* To satisfy the regression assumptions and be able to trust the results, **the residuals should have a constant variance**.
* **Important features:** hour:6,13,10, month: 3,12,snowfall

##**Random Forest**

In [ ]:
#import the packages
from sklearn.ensemble import RandomForestRegressor
# Create an instance of the RandomForestRegressor
rf_model = RandomForestRegressor()
rf_model.fit(x_train,y_train)

In [ ]:
#Getting the metric value
metric_value(rf_model,x_train,y_train,x_test,y_test)

In [ ]:
#feature importance
feature_importance(rf_model)

* **Random Forest Model**
* **R2 score of Train data : 0.99**
* **adjusted R2 score of Train data : 0.99**
* **R2 score of Test data : 0.934**
* **adjusted R2 score of Test data : 0.933**
* **Performance of data: Better**
* To satisfy the regression assumptions and be able to trust the results, **the residuals should have a constant variance**.
* **Important features:** hour:10,16,23, month:3,10, snowfall

Here, we can see that, **Functioning Day, Temperature, and Humidity** has much **higher relevance** on the counting of bike Renting. means they are most important features which affects on rented bike count in the **Random Forest model.**

##**Gradient Boosting**

In [ ]:
#import the packages
from sklearn.ensemble import GradientBoostingRegressor
# Create an instance of the GradientBoostingRegressor
gradient_boosting_regressor = GradientBoostingRegressor()
gradient_boosting_regressor.fit(x_train,y_train)

In [ ]:
#Getting the metric value
metric_value(gradient_boosting_regressor,x_train,y_train,x_test,y_test)

In [ ]:
#feature importance
feature_importance(gradient_boosting_regressor)

* **Gradient Boosting Model**
* **R2 score of Train data : 0.91**
* **adjusted R2 score of Train data : 0.90**
* **R2 score of Test data : 0.90**
* **adjusted R2 score of Test data : 0.90**
* **Performance of data: Better**
* To satisfy the regression assumptions and be able to trust the results, **the residuals should have a constant variance**.
* **Important features:** hour:1,7,11, month:8,12, windspeed, season_summer


###**Gradient Boosting with GridSearchCV**

**Provide the range of values for chosen hyperparameters**

In [ ]:
# Number of trees
n_estimators = [50,80,100]

# Maximum depth of trees
max_depth = [4,6,8,10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [30,40,50]

# HYperparameter Grid
param_dict = {'n_estimators' : n_estimators,
              'max_depth' : max_depth,
              'min_samples_leaf' : min_samples_leaf}

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create an instance of the GradientBoostingRegressor
gb_model = GradientBoostingRegressor()

# Grid search
gradient_boosting_gridsearchcv = GridSearchCV(estimator=gb_model,param_grid = param_dict,cv = 5)
gradient_boosting_gridsearchcv.fit(x_train,y_train)

In [ ]:
gb_optimal_model = gradient_boosting_gridsearchcv.best_estimator_

In [ ]:
#Getting the metric value
metric_value(gb_optimal_model,x_train,y_train,x_test,y_test)

In [ ]:
#feature importance
feature_importance(gb_optimal_model)

* **Gradient Boosting with GridSearchCV Model**
* **R2 score of Train data : 0.89**
* **adjusted R2 score of Train data : 0.89**
* **R2 score of Test data : 0.88**
* **adjusted R2 score of Test data : 0.88**
* **Performance of data: Best**
* To satisfy the regression assumptions and be able to trust the results, **the residuals should have a constant variance**.
* **Important features:** hour:10,23, month:3,6,10,12, snowfall, holiday_no holiday



##**XgBoost with RandomizedSearchCV**

In [ ]:
#Import Library
from sklearn.model_selection import RandomizedSearchCV
import xgboost

In [ ]:
#hyperparameter optimization
params = {'learning_rate' : [0.05,0.1,0.15,0.2,0.25,0.3],
          'max_depth' : [3,4,5,6,8,10,12,15],
          'min_child_weight' : [1,3,5,7],
          'gamma' : [0,0.1,0.2,0.3,0.4],
          'colsample_bytree' : [0.3,0.4,0.5,0.7]
}

In [ ]:
xgb_regressor = xgboost.XGBRegressor()
random_search = RandomizedSearchCV(xgb_regressor, param_distributions=params, n_iter=5, scoring='neg_mean_squared_error',n_jobs=-1,cv=5,verbose=3)

In [ ]:
random_search.fit(x_train,y_train)

In [ ]:
#Getting best estimator
xgb_model = random_search.best_estimator_
xgb_model

In [ ]:
#Getting the metric value
metric_value(xgb_model,x_train,y_train,x_test,y_test)

In [ ]:
#feature importance
feature_importance(xgb_model)

* **XgBoost with RandomisedSearchCV Model**
* **R2 score of Train data : 0.98**
* **adjusted R2 score of Train data : 0.98**
* **R2 score of Test data : 0.94**
* **adjusted R2 score of Test data : 0.93**
* **Performance of data: Best**
* To satisfy the regression assumptions and be able to trust the results, **the residuals should have a constant variance**.
* **Important features:** hour:1,11, month:8,12, snowfall, holiday_no_holiday, season_spring, weekdays_weekend

#**Result**

In [ ]:
from prettytable import PrettyTable

# Specify the Column Names while initializing the Table
myTable = PrettyTable(["Model_Name",'R^2 of train datset',
                       "Adjusted R^2 of train datset",
                       'R^2 of test datset', "Adjusted R^2 of test datset"])

# Add rows
myTable.add_row(["Linear Regression","86%","86%",'86%','86%'])
myTable.add_row(["Ridge Regression","86%","86%",'86%','86%'])
myTable.add_row(["Decision Tree with GridSearchCV","94%","93%",'88%','88%'])
myTable.add_row(["Random Forest", "99%","99%",'93%','93%'])
myTable.add_row(["Gradient Boosting", "89%","89%",'88%','88%'])
myTable.add_row(["Gradient Boosting with GridSearchCV", "96%","96%",'93%','93%'])
myTable.add_row(["XgBoost Regressor with RandomisedSearchCV", "98%","98%",'94%','93%'])

print(myTable)

• **No overfitting** is seen.

• **Random forest Regressor, Gradient Boosting gridsearchcv, XgBoost Regressor with RandomizedSearchCV** gives the highest R2 score.

• Feature Importance value for Random Forest, Gradient Boosting, XgBoost are different.

• We can deploy **Random Forest, Gradient Boosting with GridSearchCV, XgBoost with RandomizedSearchCV** model.

#**Model Explainability by LIME**

In [ ]:
# Extract features
float_columns=[]
cat_columns=[]
int_columns=[]

In [ ]:
# Putting features into respective float, cat , int list.
for i in x.columns:
    if x[i].dtype == 'float' :
        float_columns.append(i)
    elif x[i].dtype == 'int64':
        int_columns.append(i)
    elif x[i].dtype == 'object':
        cat_columns.append(i)

In [ ]:
train_cat_features = x[cat_columns]
train_float_features = x[float_columns]
train_int_features = x[int_columns]

In [ ]:
!pip install lime

In [ ]:
import lime
import lime.lime_tabular
from __future__ import print_function

In [ ]:
# Create the LIME Explainer
explainer = lime.lime_tabular.LimeTabularExplainer(feature_names = x.columns,
                                                  training_data = np.array(x_train),
                                                  mode='regression')

In [ ]:
# Get the explanation for RandomForest
exp = explainer.explain_instance(data_row = x_test.iloc[24], predict_fn = rf_model.predict)
exp.show_in_notebook(show_table=True)

In [ ]:
# Get the explanation for Gradient Boosting with GridSearchCV
exp = explainer.explain_instance(data_row = x_test.iloc[24], predict_fn = gradient_boosting_gridsearchcv.predict)
exp.show_in_notebook(show_table=True)

In [ ]:
# Get the explanation for XGBoost with GridSearchCV
exp = explainer.explain_instance(data_row = x_test.iloc[24], predict_fn = xgb_model.predict)
exp.show_in_notebook(show_table=True)

#**Summary and conclusions**

###**EDA Summary**
* **Count of Rented Bike(dependent variable) : positively skewed**
* **Normally distributed attributes:**
              - Temperature
              - Humidity.
* **Positively skewed attributes:**
              - Wind
              - Solar Radiation
              - Snowfall
              - Rainfall
* **Negatively skewed attributes:**
              - visibility
* **positively correlated variables to the bike rent are :**
              - Temperature
              - Dew Point Temperature
              - Solar Radiation
              - Hour

* **And most negatively correlated variables are:**
              - Humidity
              - Rainfall
              - Weekdays or Weekends
* The number of bikes rented is on average higher during the rush hours.i.e. at 6 p.m. to 8 p.m.
* The **rented bike counts is higher during the summer and lowest during the winter**.
* The **rented bike count is higher on working days than on non-working days**.
* On a **non-functioning day, no bikes are rented** in all the instances of the data.
* The number of bikes rented on average remains constant throughout Monday - Saturday, it dips on Sunday, and on average, the **rented bike counts is lower on weekends than on weekdays**.
* On regular days, the demand for the bikes is higher during rush hours. **On holidays or weekends, the demand is comparatively lower in the mornings, and is higher in the afternoons**.

**Conclusion after using different models:**

      - No overfitting is seen.
      - Random forest Regressor, Gradient Boosting gridsearchcv, XbBoost Regressor with GridSearchCV gives the highest R2 score .
      - Feature Importance value for Random Forest, Gradient Boosting, XgBoost are different.
      - We can deploy Random Forest, Gradient Boosting with  GridSearchCV, XgBoost with RandomizedSearchCV model.

**Important Feature which affect the most in the rented bike count:**

    * Functioning Day
    * Winter Season
    * 18th Hour i.e. 6 p.m.
    * Temperature
    * Humidity

**Features which impact Negatively on Rented Bike Count:**  

    * Hour 4 i.e. 4 a.m.
    * Hour 5 i.e. 5 a.m.
    * Hour 3 i.e. 3 a.m.
    * Winter Season
    * Rainfall
    * weekdays_weekend

**Features which impact Positively on Rented Bike Count:**

    * Hour 18 i.e. 6 p.m.
    * Hour 19 i.e. 7 p.m.
    * Hour 8 i.e. 8 a.m.
    * Hour 21 i.e. 9 p.m.
    * Hour 20  i.e. 8 p.m.



However, this is not the ultimate end. As **this data is time dependent**, the values of features like **Temperature, Windspeed, Solar Radiation etc., will not always be consistent.** Therefore, there will be scenarios where the model might not perform well. As Machine learning is an exponentially evolving field, we will have to be prepared for all contingencies and also keep checking our model from time to time. Therefore, having a quality knowledge and keeping pace with the ever evolving ML field would surely help one to stay a step ahead in future.